In [1]:
from transformers import CONFIG_MAPPING, BertForMaskedLM, BertModel, BertPreTrainedModel, BertTokenizerFast
import torch
import torch.nn as nn
import torch.nn.functional as F
import pdb
from pl_model.barlowbert_models import SentenceBert, BarlowBert
from pl_model.barlowbert_dm import DataCollatorForBarlowBertWithMLM
from collections import namedtuple
from datasets import load_from_disk

In [2]:
%load_ext autoreload
%autoreload 2

In [49]:
Args = namedtuple('Args',['batch_size','lambda_out','lambda_in','lambda_out_in','mlm_weight','projector', 'do_mlm'])
args = Args(16,
            0.2,
            0.001,
            1.0, #corr loss is more important
            0.5,
            '128-128-128',
            False)
config = CONFIG_MAPPING['bert'].from_pretrained('bert-base-uncased')

In [53]:
model = BarlowBert(config,args)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
# corpus = load_from_disk('/mounts/data/proj/jabbar/barlowbert/bookcorpus_20mil_128/')

In [7]:
corpus.set_format('torch')

In [8]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')#, padding=True,truncation=True)

In [9]:
collator = DataCollatorForBarlowBertWithMLM(tokenizer=tokenizer)

In [10]:
input_1 = tokenizer("The capital of France is [MASK].", return_tensors="pt",padding='max_length',truncation=True)#,return_special_tokens_mask=True,max_length=128)
input_2 = tokenizer("I am going to Berlin.", return_tensors="pt",padding='max_length',truncation=True,return_special_tokens_mask=True,max_length=128)

In [11]:
dataloader = torch.utils.data.DataLoader(corpus,batch_size=4,collate_fn=collator)

In [12]:
batch = next(iter(dataloader))

In [13]:
batch[0].keys()

dict_keys(['attention_mask', 'input_ids', 'mlm_input_ids', 'mlm_labels'])

In [54]:
out_2=model(batch)

In [55]:
out_2

{'loss': tensor(13.6726, grad_fn=<AddBackward0>),
 'sim_loss': tensor(0.2806, grad_fn=<AddBackward0>),
 'corr_loss': tensor(13.3919, grad_fn=<AddBackward0>)}

In [24]:
out_2

{'loss': tensor(538.1262, grad_fn=<AddBackward0>),
 'sim_loss': tensor(2.2527, grad_fn=<AddBackward0>),
 'corr_loss': tensor(535.3735, grad_fn=<AddBackward0>)}

In [17]:
model.train();

In [18]:
out_1= model.model(**batch[0])
out_12= model.model(**batch[1])

In [19]:
out_1['projection']-out_12['projection']

tensor([[-1.2856e-02,  1.1372e-03,  6.1142e-01, -2.4056e-03, -4.5697e-01,
         -4.9410e-02, -4.5388e-02, -1.0168e-01,  4.9248e-01,  3.1437e-01,
          1.5531e-02,  2.6873e-02, -2.4918e-01, -2.6895e-01, -4.0993e-02,
         -1.0562e-02, -5.1190e-02,  3.5261e-01, -2.3962e-01,  1.7553e-01,
         -3.2062e-02, -3.0016e-01,  1.1416e-02,  3.7877e-01, -4.4914e-01,
         -2.9914e-01,  4.1602e-01, -2.3996e-01, -5.7723e-02,  4.1246e-02,
         -1.2264e-01, -1.0458e-01, -2.2947e-01,  9.0754e-02, -2.6788e-01,
         -7.3850e-02, -1.1497e-01,  8.5765e-01, -1.5307e-01,  1.4364e-01,
          2.2260e-01,  1.4950e-01,  2.2769e-01, -6.9143e-02,  4.2038e-02,
         -5.4300e-01, -5.2231e-02,  1.6750e-01,  1.1889e-01, -5.9665e-02,
          5.0301e-02, -2.5340e-02,  8.5801e-02, -7.5178e-02, -1.2653e-02,
         -1.1717e-01,  3.9630e-01,  1.0276e-01, -6.7161e-02,  3.7028e-02,
         -3.0291e-02, -1.7160e-01, -5.6745e-01, -9.7273e-02, -3.6912e-01,
          2.4252e-01, -4.9061e-02,  3.

In [20]:
out_1.keys()

dict_keys(['projection'])

In [25]:
args.lambda_out_in

0.5